# SnapShot and SnapRestore with NetApp DataOps Toolkit for Kuberenetes

本デモではNetApp DataOps Toolkitを使用して、永続ボリューム（K8s PersistentVolume）内のデータを高速にバックアップ・リストアできることを確認します。  
NetApp DataOps Toolkitはオープンソースソフトウェアであり、利用方法も含めて下記GitHub上で公開されています。  
https://github.com/NetApp/netapp-dataops-toolkit/tree/main

In [2]:
! python3 -m pip install netapp-dataops-k8s

  Using cached netapp_dataops_k8s-2.4.0-py3-none-any.whl (45 kB)
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached func_timeout-4.3.5-py3-none-any.whl
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached dnspython-2.1.0-py3-none-any.whl (241 kB)
  Using cached urllib3-1.26.5-py2.py3-none-any.whl (138 kB)
  Using cached kubernetes-23.6.0-py2.py3-none-any.whl (1.5 MB)
  Using cached actoolkit-2.1.3-py3-none-any.whl (28 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached tabulate-0.8.9-py3-none-any.whl (25 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.2.0
    Uninstalling termcolor-2.2.0:
      Successfully uninstalled termcolor-2.2.0
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled 

## ライブラリのインポート
本デモで使用するNetApp DataOps Toolkitと汎用ライブラリをインポートします。

In [3]:
# NetApp DataOps Toolkitのインポート
from netapp_dataops.k8s import list_volumes, create_volume_snapshot, list_volume_snapshots, clone_volume

In [4]:
# 汎用ライブラリのインポート
import datetime
import os

## デモ環境の事前確認
NetApp DataOps Toolkitでバックアップを取得する前に、現在の永続ボリュームの状態を確認します。なお本ワークショップではマルチテナント化されたKubeflowの環境を想定しています。  
以下にお使いのKubeflowのテナント名（=k8s namespace）名を入力します。

In [5]:
# 利用中のテナント名（=k8s namespace）を入力
YOUR_NAMESPACE = 'yshimizu'

次にNetApp DataOps Toolkitを使用して、お使いのテナント内に払い出された永続ボリュームの一覧を確認します

In [4]:
# FSxN上に作成されたボリューム一覧の確認
volumes = list_volumes(namespace=YOUR_NAMESPACE, print_output=True)

PersistentVolumeClaim (PVC) Name    Status    Size    StorageClass    Clone    Source PVC    Source VolumeSnapshot
----------------------------------  --------  ------  --------------  -------  ------------  -----------------------
demo1-volume                        Bound     10Gi    nfs             No
demo2-volume                        Bound     10Gi    nfs             No
demo3-volume                        Bound     10Gi    nfs             No
large-vol                           Bound     1Ti     nfs-large       No
myworkspace-volume                  Bound     10Gi    nfs             No
overprovisoned-test-volume          Bound     10Gi    nfs             No
test-volume                         Bound     10Gi    nfs             No
test2-volume                        Bound     10Gi    nfs             No
test3-volume                        Bound     10Gi    nfs             No


上記出力結果からバックアップ対象のボリュームを選定し、以下の変数に代入します

In [6]:
# 上記で表示されたボリュームからバックアップ対象ボリューム（PVC）の名称を指定
PVC_NAME = 'demo1-volume'

## NetApp DataOps Toolkitを使用したボリュームの高速バックアップ
上記で確認したボリュームのバックアップを取得します。下記のコードによりNetApp DataOps ToolkitはCSI-Snapshotterを介して、volumeSnapshotの取得を行います。TridentのバックエンドストレージがNetApp ONTAPの場合、ボリューム内の実データではなく、データへのポインタ情報のみをバックアップするため大規模なボリュームであっても、バックアップ自体の容量は極めて小さく、バックアップ処理も瞬時に完了します。

In [7]:
# 取得するSnapshotの名称を生成
t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
now = datetime.datetime.now(JST)
timestamp = now.strftime('%Y%m%d%H%M%S')
SNAPSHOT_NAME = "snapshot-" + timestamp

In [8]:
# NetApp DataOps Toolkitを使用して、ボリュームのSnapshotを取得
create_volume_snapshot(namespace=YOUR_NAMESPACE, pvc_name=PVC_NAME, snapshot_name=SNAPSHOT_NAME, print_output=True)

Creating VolumeSnapshot 'snapshot-20230913140032' for PersistentVolumeClaim (PVC) 'demo1-volume' in namespace 'yshimizu'.
VolumeSnapshot 'snapshot-20230913140032' created. Waiting for Trident to create snapshot on backing storage.
Snapshot successfully created.


In [9]:
# 取得したSnapshotの一覧を確認
snap = list_volume_snapshots(namespace=YOUR_NAMESPACE, pvc_name=PVC_NAME, print_output=True)

VolumeSnapshot Name      Ready to Use    Creation Time         Source PersistentVolumeClaim (PVC)    Source JupyterLab workspace    VolumeSnapshotClass
-----------------------  --------------  --------------------  ------------------------------------  -----------------------------  ---------------------
snapshot-20230913133543  True            2023-09-13T04:35:51Z  demo1-volume                                                         csi-snapclass
snapshot-20230913140032  True            2023-09-13T05:00:39Z  demo1-volume                                                         csi-snapclass
